<a href="https://colab.research.google.com/github/iGhostlp/Albus/blob/Sin-Dientes/30-3-2023-Normalizacion_archivos_y_df_Proyecto_BBVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Armado del entorno 

In [1]:
# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [2]:
# Unzip the file
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [3]:
!readlink -f $(which java) | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [4]:
# Set up the environment for Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [5]:
# Install library for finding Spark
!pip install -q findspark

# Import the libary
import findspark

# Initiate findspark
findspark.init()

In [6]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder.master("local[*]").config('spark.sql.parquet.datetimeRebaseModeInRead','CORRECTED').getOrCreate()

# Check Spark Session Information
spark

# Importado de funciones

In [58]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, concat_ws, col, row_number, desc, collect_list, to_json, struct, year, current_date, datediff, floor, when
from pyspark.sql.window import Window


In [8]:
#Creación de un SQL context
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

/content/spark-3.3.2-bin-hadoop3/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Carga de parquets

In [9]:
df_customer = spark.read.parquet('/content/Datasets/customer_basics.snappy.parquet')
df_phones = spark.read.parquet('/content/Datasets/phones.snappy.parquet')
df_address = spark.read.parquet('/content/Datasets/address.snappy.parquet')
df_emails = spark.read.parquet('/content/Datasets/email.snappy.parquet')
df_customer_basics = spark.read.parquet('/content/Datasets/customer_basics.snappy.parquet')
df_marital_status = spark.read.parquet('/content/Datasets/marital_status_type.parquet')
df_segment_type = spark.read.parquet('/content/Datasets/segment_type.parquet')
df_customer_documents = spark.read.parquet('/content/Datasets/customer_documents.parquet')
df_address_type = spark.read.parquet('/content/Datasets/address_type.parquet')
df_gender = spark.read.parquet('/content/Datasets/gender.parquet')
df_nationality = spark.read.parquet('/content/Datasets/nationality.parquet')
df_personal_type = spark.read.parquet('/content/Datasets/personal_type.parquet')
df_phone_type = spark.read.parquet('/content/Datasets/phone_type.parquet')
df_province = spark.read.parquet('/content/Datasets/province.parquet')
df_customer_segment = spark.read.parquet('/content/Datasets/customer_segment.parquet')
df_customer_info_temp = spark.read.parquet('/content/Datasets/customer_info_temp.parquet')

### Normalización de datos

In [93]:
df_marital_ok = df_marital_status.withColumn("marital_status_type",
                                             when(df_marital_status.martial_status_short_desc == "NO INFORMA", 0)
                                             .when(df_marital_status.martial_status_short_desc == "CASADO/A", 1)
                                             .when(df_marital_status.martial_status_short_desc == "VIUDO/A", 2)
                                             .otherwise("NULO"))



In [ ]:

NO INFORMA                                        |NO INFORMA               |2016-08-11         |2016-08-11      |205030              |BTCH                   |TC4C0100           |2022-01-14 02:08:08.335|1900-01-01       |
|3                  |C                       |00001                    |CASADO/A                                          |CASADO/A                 |2016-08-11         |2016-08-11      |205030              |BTCH                   |TC4C0100           |2022-01-14 02:08:08.335|1900-01-01       |
|3                  |                        |00002                    |VIUDO/A                                           |VIUDO/A                  |2016-08-11         |2016-08-11      |205030              |BTCH                   |TC4C0100           |2022-01-14 02:08:08.335|1900-01-01       |
|3                  |C                       |00003                    |SEPARADO/A                                        |SEPARADO/A               |2016-08-11         |2016-08-11      |205030              |BTCH                   |TC4C0100           |2022-01-14 02:08:08.335|1900-01-01       |
|3                  |                        |00003                    |DIVORCIADO                                        |DIVORCIADO               |2016-08-11         |2016-08-11      |205030              |BTCH                   |TC4C0100           |2022-01-14 02:08:08.335|1900-01-01       |
|3                  |                        |00000                    |SOLTERO/A                                         |SOLTERO/A                |2016-08-11         |2016-08-11      |205030              |BTCH                   |TC4C0100           |2022-01-14 02:08:08.335|1900-01-01       |
|3                  |                        |                         |CONVIVIENT                                        |CONVIVIENT               |2016-08-11         |2016-08-11      |205030              |BTCH                   |TC4C0100           |2022-01-14 02:08:08.335|1900-01-01       |
|3                  |                        |                         |OTROS                                             |OTROS                    |2016-08-11         |2016-08-11      |205030              |BTCH                   |TC4C0100           |2022-01-14 02:08:08.335|1900-01-01       |
|3                  |                        |                         |CONV. INSC   

# Extraccion de datos


### - 24 - Extraccion de datos desde parquet, clientes y teléfonos.
####Crear un DataFrame que contenga el JOIN de la tabla t_abtq_customer_basics y tabla t_abtq_customer_phones.

In [10]:
#df_customer_phones = df_phones.join(df_customer, 'customer_id')
df_customer_phones = df_phones.join(df_customer.select(col("customer_id"), col("last_change_date").alias("l_c_d_customer")), "customer_id")


In [11]:
df_customer_phones.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- phone_use_type: string (nullable = true)
 |-- address_sequence_id: integer (nullable = true)
 |-- phone_sequence_id: integer (nullable = true)
 |-- phone_type: string (nullable = true)
 |-- phone_country_id: string (nullable = true)
 |-- prefix_phone_id: string (nullable = true)
 |-- phone_area_id: string (nullable = true)
 |-- cellphone_prefix_id: string (nullable = true)
 |-- phone_exchange_id: string (nullable = true)
 |-- phone_line_id: string (nullable = true)
 |-- phone_id: string (nullable = true)
 |-- phone_intern_id: string (nullable = true)
 |-- aditional_info_txt_desc: string (nullable = true)
 |-- primary_phone_type: string (nullable = true)
 |-- address_sequence_type: string (nullable = true)
 |-- address_town_name: string (nullable = true)
 |-- zipcode_id: string (nullable = true)
 |-- province_id: string (nullable = true)
 |-- customer_phone_status_type: string (nullable = true)
 |-- phone_status_mod_date: date (nullab

### - 25 - Extraccion de datos desde .parquet, clientes y direcciones
####Crear un DataFrame que contenga el JOIN de la tabla t_abtq_customer_basics y tabla t_abtq_customer_adress.

In [12]:
sorted(df_address.columns)

['address_country_id',
 'address_department_name',
 'address_district_name',
 'address_indoor_id',
 'address_outdoor_id',
 'address_priority_number',
 'address_relationship_type',
 'address_sequence_id',
 'address_start_date',
 'address_status_mod_date',
 'address_town_name',
 'address_verified_date',
 'address_without_number_type',
 'contact_channel_type',
 'customer_id',
 'customer_locator_verified_type',
 'delivery_contact_end_hm_date',
 'delivery_contact_start_hm_date',
 'dlvy_day_friday_type',
 'dlvy_day_monday_type',
 'dlvy_day_saturday_type',
 'dlvy_day_thursday_type',
 'dlvy_day_tuesday_type',
 'dlvy_day_wednesday_type',
 'indoor_number',
 'last_change_date',
 'last_change_hms_date',
 'last_change_terminal_id',
 'last_change_user_id',
 'long_zipcode_id',
 'normalization_date',
 'normalization_reason_name',
 'normalization_status_type',
 'normalized_level_match_number',
 'operational_load_date',
 'other_information_desc',
 'prev_address_sequence_id',
 'province_id',
 'register_u

In [13]:
df_address.show()

+-----------+-----------------------+-------------------+--------------+--------------------+------------------+---------------------------+-----------------+-------------+---------------------+--------------------+-----------------------+----------+---------------+-----------+------------------+----------------------+-------------------------+------------------+---------------------+------------------------------+-----------------------+--------------------+---------------------+------------------+-------------------------+-------------------------+------------------+-----------------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+-----------------+---------------------+
|customer_id|add

In [14]:
df_customer.show()

+-----------+-------------+-------------+--------------+---------------+--------------------+---------+----------------+-----------+-----------------------+-----------------------+----------+-------------------+-----------------+--------------------------+----------+----------------+-----------------+-------------------+-----------+----------+----------------------+--------------------+---------------------+-----------+--------------------------+----------------------------+-------------------------------+------------------------+------------------------+--------------------------+------------------------+--------------------------+----------------------------+------------------+------------------------+-------------------+-------------------+---------------------------+-----------------+-------------------+-----------------+-------------------------------+--------------------+---------------------+------------------+----------------+--------------------+--------------------------+----

In [15]:
#df_customer_address = df_address.join(df_customer, 'customer_id')
#df_customer_address = df_address.join(df_customer.select(col("customer_id"), col("last_change_date").alias("l_c_d_customer")), "customer_id")

### - 26 - Extraccion de datos desde .parquet, clientes y correos electrónicos
#### Crear un DataFrame que contenga el JOIN de la tabla t_abtq_customer_basics y tabla t_abtq_customer_email.  

In [16]:
sorted(df_emails.columns)

['address_sequence_id',
 'comments_desc',
 'customer_email_status_date',
 'customer_email_status_type',
 'customer_id',
 'email_app_type',
 'email_desc',
 'email_domain_type',
 'email_type',
 'encripted_type',
 'field_length_number',
 'last_change_date',
 'last_change_hms_date',
 'last_change_terminal_id',
 'last_change_user_id',
 'operational_load_date',
 'primary_email_type',
 'register_user_id',
 'registry_entry_date',
 'residence_type',
 'role_type']

In [17]:
#df_customer_emails = df_emails.join(df_customer, 'customer_id', how="right")
df_customer_emails = df_customer.join(df_emails.select(col("customer_id"),col("email_desc"), col("last_change_date").alias("last_change_date_e")), "customer_id")

In [18]:
sorted(df_customer_emails.columns)

['admission_date',
 'balance_closing_date',
 'bbva_cust_number',
 'bbva_family_lnk_weighted_number',
 'bbva_inhabilitation_type',
 'birth_date',
 'branch_id',
 'business_area_id',
 'campaing_id',
 'coholder_customer_id',
 'country_nationality_id',
 'credit_manager_id',
 'credit_recovery_manager_id',
 'cust_bcra_econ_activity_type',
 'customer_bcra_situation_id',
 'customer_block_bbva_date',
 'customer_block_bcra_date',
 'customer_condition_type',
 'customer_cuit_id',
 'customer_end_date',
 'customer_entry_type',
 'customer_first_block_date',
 'customer_id',
 'customer_level_access_type',
 'customer_linked_type',
 'customer_pending_notice_type',
 'customer_pep_type',
 'customer_position_vat_type',
 'customer_registration_date',
 'customer_seniority_number',
 'customer_situation_type',
 'customer_status_type',
 'customer_subject_type',
 'customer_type',
 'customer_unblock_bbva_date',
 'customer_unblock_bcra_date',
 'email_desc',
 'first_name',
 'gender_type',
 'internal_economic_activity

# Filtrar datos, reducir volumen

####- 27 - Filtrar el DataFrame de contactos telefónicos de clientes y resguardar los 3 contactos más actuales por cliente.

In [19]:
df_phones_cut = df_customer_phones.drop('contact_channel_type','wrong_phone_type','registry_entry_date','register_user_id','last_change_user_id','l_c_d_customer','last_change_hms_date','last_change_terminal_id','phone_intern_id','phone_country_id', 'aditional_info_txt_desc', 'primary_phone_type','address_sequence_type','address_town_name','zipcode_id', 'province_id','sender_application_id','normalization_status_type','normalization_reason_name','validity_start_date','validity_end_date','dlvy_day_monday_type','dlvy_day_tuesday_type','dlvy_day_wednesday_type','dlvy_day_thursday_type','dlvy_day_friday_type','dlvy_day_friday_type','dlvy_day_saturday_type','delivery_contact_start_hm_date','delivery_contact_end_hm_date','operational_load_date','normalization_date')

In [20]:
df_phones_sorted = df_phones_cut.orderBy([df_phones_cut.customer_id, desc('last_change_date')])
df_phones_sorted.show()

+-----------+--------------+-------------------+-----------------+----------+---------------+-------------+-------------------+-----------------+-------------+------------+--------------------------+---------------------+----------------+
|customer_id|phone_use_type|address_sequence_id|phone_sequence_id|phone_type|prefix_phone_id|phone_area_id|cellphone_prefix_id|phone_exchange_id|phone_line_id|    phone_id|customer_phone_status_type|phone_status_mod_date|last_change_date|
+-----------+--------------+-------------------+-----------------+----------+---------------+-------------+-------------------+-----------------+-------------+------------+--------------------------+---------------------+----------------+
|   06987700|            01|                  0|                2|         1|             54|           11|                   |             4451|         4104|  5848841581|                 DECLARADO|           2022-07-18|      2022-07-18|
|   19801715|            01|                

In [21]:
#df_phones_sorted = df_phones_sorted.select(concat(df_phones_sorted.prefix_phone_id,df_phones_sorted.phone_area_id,df_phones_sorted.phone_exchange_id,df_phones_sorted.phone_line_id).alias('Full_Phone'),'customer_id','last_change_date')

In [22]:
df_phones_sorted = df_phones_sorted.select(concat_ws('-', df_phones_sorted.prefix_phone_id,df_phones_sorted.phone_area_id,df_phones_sorted.phone_exchange_id,df_phones_sorted.phone_line_id).alias('Full_Phone'),'customer_id','last_change_date')

In [23]:
df_phones_sorted.show()

+---------------+-----------+----------------+
|     Full_Phone|customer_id|last_change_date|
+---------------+-----------+----------------+
|54-11-4451-4104|   06987700|      2022-07-18|
|54-11-4544-8655|   19801715|      2022-08-01|
|54-11-6286-0946|   21162204|      2022-09-28|
|54-11-4788-3665|   23417903|      2022-11-08|
|54-11-5656-3942|   24401530|      2022-07-04|
|54-3447-64-4382|   24935788|      2022-10-11|
|54-11-5667-3375|   25633010|      2022-10-13|
|54-11-4342-8605|   25633010|      2022-09-30|
|54-11-4341-0084|   26183611|      2022-07-21|
|54-11-4341-0084|   26183611|      2022-07-21|
|54-2323-67-7206|   26293280|      2022-07-21|
|54-2964-54-9154|   26488506|      2022-09-01|
|54-11-6705-6375|   26923214|      2022-11-22|
|54-2657-31-3231|   27008407|      2022-10-24|
|54-2657-22-6503|   27008407|      2022-10-24|
|54-11-6161-5326|   28445138|      2022-11-23|
|54-351-716-8741|   28633515|      2022-11-19|
|54-223-451-2038|   28660498|      2022-09-12|
+------------

In [24]:
window = Window.partitionBy(df_phones_sorted.customer_id).orderBy(desc(df_phones_sorted.last_change_date))

In [25]:
df_phone = df_phones_sorted.withColumn('row_num', row_number().over(window))

In [26]:
df_phone = df_phone.filter(df_phone.row_num <= 3)

In [27]:
df_phone.show()

+---------------+-----------+----------------+-------+
|     Full_Phone|customer_id|last_change_date|row_num|
+---------------+-----------+----------------+-------+
|54-11-4451-4104|   06987700|      2022-07-18|      1|
|54-11-4544-8655|   19801715|      2022-08-01|      1|
|54-11-6286-0946|   21162204|      2022-09-28|      1|
|54-11-4788-3665|   23417903|      2022-11-08|      1|
|54-11-5656-3942|   24401530|      2022-07-04|      1|
|54-3447-64-4382|   24935788|      2022-10-11|      1|
|54-11-5667-3375|   25633010|      2022-10-13|      1|
|54-11-4342-8605|   25633010|      2022-09-30|      2|
|54-11-4341-0084|   26183611|      2022-07-21|      1|
|54-11-4341-0084|   26183611|      2022-07-21|      2|
|54-2323-67-7206|   26293280|      2022-07-21|      1|
|54-2964-54-9154|   26488506|      2022-09-01|      1|
|54-11-6705-6375|   26923214|      2022-11-22|      1|
|54-2657-22-6503|   27008407|      2022-10-24|      1|
|54-2657-31-3231|   27008407|      2022-10-24|      2|
|54-11-616

In [28]:
df_pivot_phone = df_phone.groupBy('customer_id').agg(collect_list('Full_Phone').alias('last_3_changes_list'))

In [29]:
df_pivot_phone = df_pivot_phone.selectExpr('customer_id', 'last_3_changes_list[0] as Phone_1', 'last_3_changes_list[1] as Phone_2', 'last_3_changes_list[2] as Phone_3')

In [30]:
df_pivot_phone.show()

+-----------+---------------+---------------+-------+
|customer_id|        Phone_1|        Phone_2|Phone_3|
+-----------+---------------+---------------+-------+
|   06987700|54-11-4451-4104|           null|   null|
|   19801715|54-11-4544-8655|           null|   null|
|   21162204|54-11-6286-0946|           null|   null|
|   23417903|54-11-4788-3665|           null|   null|
|   24401530|54-11-5656-3942|           null|   null|
|   24935788|54-3447-64-4382|           null|   null|
|   25633010|54-11-5667-3375|54-11-4342-8605|   null|
|   26183611|54-11-4341-0084|54-11-4341-0084|   null|
|   26293280|54-2323-67-7206|           null|   null|
|   26488506|54-2964-54-9154|           null|   null|
|   26923214|54-11-6705-6375|           null|   null|
|   27008407|54-2657-22-6503|54-2657-31-3231|   null|
|   28445138|54-11-6161-5326|           null|   null|
|   28633515|54-351-716-8741|           null|   null|
|   28660498|54-223-451-2038|           null|   null|
+-----------+---------------

In [31]:
# preparar para json

In [32]:
df_phones_json = df_phones.select('customer_id','Phone_type','prefix_phone_id','phone_area_id','phone_exchange_id','phone_line_id')
#df_phones_json = df_phones.select('Phone_type','prefix_phone_id','phone_area_id','phone_exchange_id','phone_line_id')

In [33]:
df_phones_json.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- Phone_type: string (nullable = true)
 |-- prefix_phone_id: string (nullable = true)
 |-- phone_area_id: string (nullable = true)
 |-- phone_exchange_id: string (nullable = true)
 |-- phone_line_id: string (nullable = true)



In [34]:
from pyspark.sql.functions import to_json

In [35]:
df_json_concat = df_phones_json.select(concat_ws("-", df_phones_json.Phone_type, df_phones_json.prefix_phone_id, df_phones_json.phone_area_id, df_phones_json.phone_exchange_id, df_phones_json.phone_line_id).alias('Full_Phone_json'))
#df_phones_sorted = df_phones_sorted.select(concat_ws('-', df_phones_sorted.prefix_phone_id,df_phones_sorted.phone_area_id,df_phones_sorted.phone_exchange_id,df_phones_sorted.phone_line_id).alias('Full_Phone'),'customer_id','last_change_date')


In [36]:
df_json_concat.show()

+-----------------+
|  Full_Phone_json|
+-----------------+
|1-54-2901-43-1039|
|3-54-3442-51-6913|
|1-54-3442-42-6301|
|1-54-341-499-4411|
|3-54-299-553-4316|
|3-54-3571-54-1998|
|3-54-294-459-8781|
|1-54-2657-42-2205|
|3-54-341-308-6577|
|1-54-2966-42-6622|
|3-54-379-517-4233|
|3-54-11-3422-9301|
|3-54-299-468-6952|
|1-54-11-4131-1252|
|3-54-381-534-6781|
|1-54-266-444-9170|
|3-54-264-551-0219|
|3-54-2657-32-7514|
|3-54-2657-39-0843|
|3-54-11-5701-3654|
+-----------------+
only showing top 20 rows



#####Tabla filtrada de phones

In [37]:
df_pivot_phone = df_pivot_phone.na.fill('---')
df_pivot_phone.show()

+-----------+---------------+---------------+-------+
|customer_id|        Phone_1|        Phone_2|Phone_3|
+-----------+---------------+---------------+-------+
|   06987700|54-11-4451-4104|            ---|    ---|
|   19801715|54-11-4544-8655|            ---|    ---|
|   21162204|54-11-6286-0946|            ---|    ---|
|   23417903|54-11-4788-3665|            ---|    ---|
|   24401530|54-11-5656-3942|            ---|    ---|
|   24935788|54-3447-64-4382|            ---|    ---|
|   25633010|54-11-5667-3375|54-11-4342-8605|    ---|
|   26183611|54-11-4341-0084|54-11-4341-0084|    ---|
|   26293280|54-2323-67-7206|            ---|    ---|
|   26488506|54-2964-54-9154|            ---|    ---|
|   26923214|54-11-6705-6375|            ---|    ---|
|   27008407|54-2657-22-6503|54-2657-31-3231|    ---|
|   28445138|54-11-6161-5326|            ---|    ---|
|   28633515|54-351-716-8741|            ---|    ---|
|   28660498|54-223-451-2038|            ---|    ---|
+-----------+---------------

####- 28 - Filtrar el DataFrame de direcciones de clientes y resguardar los 3 contactos más actuales por cliente.

In [38]:
#antes de hacer el join, voy a hacer un nuevo dataframe, con los campos que se solicitan en el Ticket 32: --- Calle, Número, Piso, Depto, Localidad, Provincia, Código postal ---

In [39]:
df_address_cut = df_address.select(col("customer_id"),col("street_name"),col("address_outdoor_id"),col("address_indoor_id"),col("indoor_number"),col("address_department_name"),col("province_id"),col("zipcode_id"),col("last_change_date").alias("last_change_date_a"))

In [40]:
df_address_cut.toPandas()

,customer_id,street_name,address_outdoor_id,address_indoor_id,indoor_number,address_department_name,province_id,zipcode_id,last_change_date_a
0,00000003,AV PRES BARTOLOME MITRE,0001500,,,AVELLANEDA,02,01870,2011-09-07
1,00000009,CNEL AREVALO,0002391,,,CAPITAL FEDERAL,01,01425,2014-07-22
2,00000009,VENEZUELA 538,23,1,CAPITAL,112233114455,11,11111,2017-04-18
3,00000011,AV FELIX DE OLAZABAL,0005392,1,C,CAPITAL FEDERAL,01,01431,2007-11-02
4,00000027,HEROES DE MALVI,0001944,00,,112233114455,02,01824,1998-12-18
...,...,...,...,...,...,...,...,...,...
2495,00017540,AV DIRECTORIO,0004901,PB,,CAPITAL FEDERAL,01,01440,2009-01-26
2496,00017558,CAJARAVILLA,0004704,,,CAPITAL FEDERAL,01,01407,2009-01-26
2497,00017561,ALMEYRA,0003012,,,GENERAL SAN MARTIN,02,01650,2009-01-26
2498,00017568,ARAOZ,0002083,,,MALVINAS ARGENTINAS,02,01613,2009-01-26


In [41]:
df_address_sorted = df_address_cut.orderBy([df_address_cut.customer_id, desc('last_change_date_a')])


In [42]:
#df_address_sorted = df_address_sorted.select('customer_id', concat(df_address_sorted.street_name,df_address_sorted.address_outdoor_id,df_address_sorted.address_indoor_id,df_address_sorted.indoor_number).alias('Full_Address'),'address_district_name','address_town_name','address_department_name','zipcode_id','long_zipcode_id','last_change_date')
df_address_sorted = df_address_sorted.select('customer_id', concat_ws('-', df_address_sorted.street_name,df_address_sorted.address_outdoor_id,df_address_sorted.address_indoor_id,df_address_sorted.indoor_number,df_address_sorted.address_department_name,df_address_sorted.province_id,df_address_sorted.zipcode_id).alias('Full_Address'),'last_change_date_a')

In [43]:
df_address_sorted.toPandas()

,customer_id,Full_Address,last_change_date_a
0,00000003,AV PRES BARTOLOME MITRE -0001500- - ...,2011-09-07
1,00000009,VENEZUELA 538 -23 -1 -CAPI...,2017-04-18
2,00000009,CNEL AREVALO -0002391- - ...,2014-07-22
3,00000011,AV FELIX DE OLAZABAL -0005392-1 -C ...,2007-11-02
4,00000027,AVENIDA DE MAYO -0001465-2 -AA ...,2010-01-25
...,...,...,...
2495,27282128,SAAVEDRA -0002548- - ...,2011-07-01
2496,27408820,ELOY FUENTES -0000041- - ...,2021-12-14
2497,27564067,ZEBALLOS -0000900- - ...,2014-02-17
2498,27568614,AVDA 25 -0002500- - ...,2020-11-25


In [44]:
window = Window.partitionBy(df_address_sorted.customer_id).orderBy(desc(df_address_sorted.last_change_date_a))

In [45]:
df_address = df_address_sorted.withColumn('row_num', row_number().over(window))

In [46]:
df_address = df_address.filter(df_address.row_num <= 3)
df_address.toPandas()

,customer_id,Full_Address,last_change_date_a,row_num
0,00000003,AV PRES BARTOLOME MITRE -0001500- - ...,2011-09-07,1
1,00000009,VENEZUELA 538 -23 -1 -CAPI...,2017-04-18,1
2,00000009,CNEL AREVALO -0002391- - ...,2014-07-22,2
3,00000011,AV FELIX DE OLAZABAL -0005392-1 -C ...,2007-11-02,1
4,00000027,AVENIDA DE MAYO -0001465-2 -AA ...,2010-01-25,1
...,...,...,...,...
2478,27282128,SAAVEDRA -0002548- - ...,2011-07-01,2
2479,27408820,ELOY FUENTES -0000041- - ...,2021-12-14,1
2480,27564067,ZEBALLOS -0000900- - ...,2014-02-17,1
2481,27568614,AVDA 25 -0002500- - ...,2020-11-25,1


In [47]:
df_pivot_address = df_address.groupBy('customer_id').agg(collect_list('Full_Address').alias('last_3_changes_list'))

In [48]:
df_pivot_address = df_pivot_address.selectExpr('customer_id', 'last_3_changes_list[0] as Address_1', 'last_3_changes_list[1] as Address_2', 'last_3_changes_list[2] as Address_3')


In [49]:
df_pivot_address = df_pivot_address.na.fill('---')
df_pivot_address.toPandas()

,customer_id,Address_1,Address_2,Address_3
0,00000003,AV PRES BARTOLOME MITRE -0001500- - ...,---,---
1,00000009,VENEZUELA 538 -23 -1 -CAPI...,CNEL AREVALO -0002391- - ...,---
2,00000011,AV FELIX DE OLAZABAL -0005392-1 -C ...,---,---
3,00000027,AVENIDA DE MAYO -0001465-2 -AA ...,PJE JACHAL -0003547- - ...,HEROES DE MALVI -0001944-00 - ...
4,00000028,NECOCHEA -0004653-35 -1 ...,---,---
...,...,...,...,...
2395,27282128,SAAVEDRA -0002548- - ...,SAAVEDRA -0002548- - ...,---
2396,27408820,ELOY FUENTES -0000041- - ...,---,---
2397,27564067,ZEBALLOS -0000900- - ...,---,---
2398,27568614,AVDA 25 -0002500- - ...,---,---


####- 29 - Filtrar el DataFrame de correos electrónicos de clientes  y resguardar los 3 contactos más actuales por cliente.

In [ ]:
#df_emails_cut = df_customer_emails.drop('role_type', 'email_type','address_sequence_id','residence_type','primary_email_type','email_domain_type','encripted_type','field_length_number','comments_desc','customer_email_status_type','email_app_type','register_user_id','last_change_user_id','last_change_hms_date','last_change_terminal_id','operational_load_date','customer_email_status_date','registry_entry_date')
df_emails_cut = df_customer_emails.select('customer_id','email_desc','last_change_date_e')

In [ ]:
df_customer_emails.show()

+-----------+-------------+-----------+--------------+---------------+--------------------+---------+----------------+-----------+-----------------------+-----------------------+----------+-------------------+-----------------+--------------------------+---------+----------------+----------+-------------------+-----------+----------+----------------------+--------------------+---------------------+-----------+--------------------------+----------------------------+-------------------------------+------------------------+------------------------+--------------------------+------------------------+--------------------------+----------------------------+------------------+------------------------+-------------------+-------------------+---------------------------+-----------------+-------------------+-----------------+-------------------------------+--------------------+---------------------+------------------+----------------+--------------------+--------------------------+--------------

In [ ]:
df_emails_sorted = df_emails_cut.orderBy([df_emails_cut.customer_id, desc('last_change_date_e')])

df_emails_sorted.show()

+-----------+----------+------------------+
|customer_id|email_desc|last_change_date_e|
+-----------+----------+------------------+
+-----------+----------+------------------+



In [ ]:
window = Window.partitionBy(df_emails_sorted.customer_id).orderBy(desc(df_emails_sorted.last_change_date_e))

In [ ]:
df_email = df_emails_sorted.withColumn('row_num', row_number().over(window))

In [ ]:
df_email = df_email.filter(df_email.row_num <= 3)

In [ ]:
df_email.toPandas()

,customer_id,email_desc,last_change_date_e,row_num


In [ ]:
df_pivot_email = df_email.groupBy('customer_id').agg(collect_list('email_desc').alias('last_3_changes_list'))

In [ ]:
df_pivot_email = df_pivot_email.selectExpr('customer_id', 'last_3_changes_list[0] as Email_1', 'last_3_changes_list[1] as Email_2', 'last_3_changes_list[2] as Email_3')

In [ ]:
df_pivot_email.toPandas()

,customer_id,Email_1,Email_2,Email_3


In [ ]:
df_pivot_email = df_pivot_email.na.fill('---')
df_pivot_email.toPandas()

,customer_id,Email_1,Email_2,Email_3


#Enrequecimiento de datos.

####- 30 - Agregar una nueva columna a los DataFrame de contactos, indicando el contact_type según corresponda (address, email, phone)

In [ ]:
df_contact_types = df_phones_contact_col.join(df_address_contact_col, 'customer_id', how='full')\
.join(df_emails_contact_col, 'customer_id', how='full')

NameError: ignored

In [ ]:
df_contact_types.select('customer_id','contact_type_phones','contact_type_address','contact_type_emails').show()

NameError: ignored

Join tablas pivot

In [ ]:
df_contactos = df_pivot_phone.join(df_pivot_email, "customer_id") \
                   .join(df_pivot_address, "customer_id")

In [ ]:
sorted(df_contactos.columns)

['Address_1',
 'Address_2',
 'Address_3',
 'Email_1',
 'Email_2',
 'Email_3',
 'Phone_1',
 'Phone_2',
 'Phone_3',
 'customer_id']

In [ ]:
df_contactos.show()

+-----------+-------+-------+-------+-------+-------+-------+---------+---------+---------+
|customer_id|Phone_1|Phone_2|Phone_3|Email_1|Email_2|Email_3|Address_1|Address_2|Address_3|
+-----------+-------+-------+-------+-------+-------+-------+---------+---------+---------+
+-----------+-------+-------+-------+-------+-------+-------+---------+---------+---------+



####- 31 - Agregar una nueva columna al DataFrame de contactos telefónicos de clientes, resguardando el contacto en formato json contenido en string, con los datos: Phone_type (mobile, landline ), Código país., Código de Área, Número teléfono.

In [ ]:
df_phone_contact = df_phone_contact.withColumn("phone_contact", to_json(struct(df_phone_contact.phone_type, df_phone_contact.phone_country_id, df_phone_contact.prefix_phone_id, df_phone_contact.phone_area_id, df_phone_contact.cellphone_prefix_id, df_phone_contact.phone_exchange_id, df_phone_contact.phone_line_id)))

NameError: ignored

In [ ]:
df_phone_contact.show(truncate=False)

NameError: ignored

####- 32 - Agregar una nueva columna al DataFrame de direcciones de clientes, resguardando el contacto en formato json contenido en string, con los datos: Calle, Número, Piso, Depto, Localidad, Provincia, Código postal

####- 33 - Combinar los DataFrame de contactos telefónicos de clientes, direcciones de clientes y email de clientes en uno solo.

In [ ]:
#BBVAW2T2-33


df_contactos = df_pivot_phone.join(df_pivot_email, "customer_id") \
                   .join(df_pivot_address, "customer_id")

In [ ]:
sorted(df_contactos.columns)

['Address_1',
 'Address_2',
 'Address_3',
 'Email_1',
 'Email_2',
 'Email_3',
 'Phone_1',
 'Phone_2',
 'Phone_3',
 'customer_id']

In [ ]:
df_contactos.printSchema()

root
 |-- customer_id: string (nullable = false)
 |-- Phone_1: string (nullable = false)
 |-- Phone_2: string (nullable = false)
 |-- Phone_3: string (nullable = false)
 |-- Email_1: string (nullable = false)
 |-- Email_2: string (nullable = false)
 |-- Email_3: string (nullable = false)
 |-- Address_1: string (nullable = false)
 |-- Address_2: string (nullable = false)
 |-- Address_3: string (nullable = false)



In [ ]:
df_contactos_1 = df_contactos.select('customer_id','Phone_1','Email_1','Address_1')

In [ ]:
df_contactos_1.createOrReplaceTempView("contactos_temp")



In [ ]:
df_contactos_1.show()

+-----------+-------+-------+---------+
|customer_id|Phone_1|Email_1|Address_1|
+-----------+-------+-------+---------+
+-----------+-------+-------+---------+



# Creacion de vistas temporales.

####- 35 - Generar una vista temporal a partir del DataFrame de contactos.

In [51]:
df_contactos_1.createTempView('tw_contactos_1')

NameError: ignored

####- 36 - Generar una vista temporal a partir del archivo t_abtq_customer_basics.

In [52]:
df_customer_basics.createOrReplaceTempView('tw_customer_basics')

####- 37 - Generar una vista temporal a partir del archivo t_acog_marital_status_type.

In [53]:
df_marital_status.createOrReplaceTempView('tw_marital_status')

####- 38 - Generar una vista temporal a partir del archivo t_acog_nationality.

In [54]:
df_nationality.createOrReplaceTempView('tw_nationality')

# Querys en spark SQL.

####- 39 - Generar un público objetivo (1) que cumpla los siguientes puntos:
* Cliente
* Antigüedad superior a 5 años
* Asalariado Fijo.
* Estado civil: Casado


In [ ]:
publico_obj1 = sqlContext.sql('select customer_id, customer_condition_type, marital_status_type,job_type,floor(datediff(current_date(), admission_date)/365) as years_diff  FROM tw_customer_basics where floor(datediff(current_date(), admission_date)/365) > 5 AND marital_status_type = 01 AND job_type = 001 AND customer_condition_type = 1')

In [ ]:
publico_obj1.show()

+-----------+-----------------------+-------------------+--------+----------+
|customer_id|customer_condition_type|marital_status_type|job_type|years_diff|
+-----------+-----------------------+-------------------+--------+----------+
|   06916693|                      1|                  1|     001|        16|
|   20706165|                      1|                  1|     001|         8|
|   02997060|                      1|                  1|     001|        12|
|   04484525|                      1|                  1|     001|        31|
|   02337659|                      1|                  1|     001|        13|
|   00888642|                      1|                  1|     001|        30|
|   06452202|                      1|                  1|     001|        18|
|   08792054|                      1|                  1|     001|        14|
|   04154408|                      1|                  1|     001|        24|
|   07007301|                      1|                  1|     00

####- 40 -  Generar un público objetivo (2) que cumpla los siguientes puntos:
* Potencial Cliente
* Sexo Femenino
* Entre 30 y 45 años de Edad.
* Nacionalidad No Argentina

In [ ]:
publico_obj2 = sqlContext.sql('select customer_id, country_nationality_id, customer_condition_type, gender_type,floor(datediff(current_date(), birth_date)/365) as customer_age  FROM tw_customer_basics WHERE floor(datediff(current_date(), birth_date)/365) > 30 AND floor(datediff(current_date(), birth_date)/365) < 45 AND gender_type = "F" AND customer_condition_type = 1 AND country_nationality_id <> 80')


In [ ]:
publico_obj2.show()

+-----------+----------------------+-----------------------+-----------+------------+
|customer_id|country_nationality_id|customer_condition_type|gender_type|customer_age|
+-----------+----------------------+-----------------------+-----------+------------+
|   08576564|                    69|                      1|          F|          44|
|   22184167|                    83|                      1|          F|          33|
+-----------+----------------------+-----------------------+-----------+------------+



####- 41 - Generar un público objetivo (3) que cumpla los siguientes puntos:
* Potencial Cliente
* Sexo Masculino.
* Mayor a 25 años.
* Estado civil Soltero

In [ ]:
publico_obj3 = sqlContext.sql('select customer_id, customer_condition_type, gender_type, marital_status_type, floor(datediff(current_date(), birth_date)/365) as customer_age  FROM tw_customer_basics WHERE floor(datediff(current_date(), birth_date)/365) > 25 AND gender_type = "M" AND marital_status_type = 5')
#AND customer_condition_type = 1

In [ ]:
publico_obj3.show()

+-----------+-----------------------+-----------+-------------------+------------+
|customer_id|customer_condition_type|gender_type|marital_status_type|customer_age|
+-----------+-----------------------+-----------+-------------------+------------+
|   26745469|                      1|          M|                  5|          65|
|   19963134|                      1|          M|                  5|          74|
|   03099590|                      1|          M|                  5|          79|
|   04762676|                      1|          M|                  5|          47|
|   05483567|                      1|          M|                  5|          45|
|   02169869|                      1|          M|                  5|          57|
|   26817297|                      8|          M|                  5|          27|
|   03613890|                      8|          M|                  5|          77|
|   06771433|                      1|          M|                  5|          45|
|   

####- 42 - Generar un público objetivo (4) que cumpla los siguientes puntos:
* Cliente
* Edad superior a 52 años
* Nacionalidad Argentina
* Estado civil Viudo

In [55]:
publico_obj4 = sqlContext.sql('select customer_id, customer_condition_type, country_nationality_id, marital_status_type, floor(datediff(current_date(), birth_date)/365) as customer_age  FROM tw_customer_basics WHERE floor(datediff(current_date(), birth_date)/365) > 52 AND country_nationality_id = 80 AND marital_status_type = 2 AND customer_condition_type = 1')


In [56]:
publico_obj4.show()

+-----------+-----------------------+----------------------+-------------------+------------+
|customer_id|customer_condition_type|country_nationality_id|marital_status_type|customer_age|
+-----------+-----------------------+----------------------+-------------------+------------+
|   04191069|                      1|                    80|                  2|          89|
|   03925479|                      1|                    80|                  2|          57|
|   04161701|                      1|                    80|                  2|          87|
|   27541480|                      1|                    80|                  2|          87|
|   04374298|                      1|                    80|                  2|          57|
|   05353458|                      1|                    80|                  2|         102|
|   05812127|                      1|                    80|                  2|          98|
|   08713474|                      1|                    80|

# Carga de datos, generacion de archivos .csv
#### Los archivos deben cumplir los siguientes requerimientos: 
* Contener cabecera. 
* Separador “|”. 
* Máximo de 1000 registros por archivo (si el público objetivo tiene más de 1000 registros, se deberá crear más de un archivo)
* Ser guardados en HDFS.

#### - 43 -  Generar archivos .csv a partir de los público objetivo 1. 


In [ ]:
# Importar librerías necesarias
from pyspark.sql import SparkSession

# Crear sesión de Spark
spark = SparkSession.builder.appName('Export to CSV').getOrCreate()

# Cargar DataFrame
df = spark.read.format('csv').option('header', 'true').load('ruta/al/archivo')

# Dividir DataFrame en bloques de 1000 registros
n = 1000
dfs = [df.limit(n).offset(i*n) for i in range(int(df.count()/n)+1)]

# Exportar cada bloque a un archivo CSV
for i, df_part in enumerate(dfs):
    # Definir ruta y nombre de archivo
    filename = 'ruta/en/hdfs/archivo_{}.csv'.format(i)
    
    # Exportar DataFrame a CSV con separador '|'
    df_part.write.format('csv').option('header', 'true').option('delimiter', '|').mode('overwrite').save(filename)


#### - 44 - Generar archivos .csv a partir del público objetivo 2. 

####- 45 - Generar archivos .csv a partir del público objetivo 3. 

####- 46 - Generar archivos .csv a partir del público objetivo 4. 

# Extraccion de datos desde .csv, públicos objetivo

#### - 47 - Cargar los datos de los publicos generados en un DataFrame.

# Calculo de agregaciones.

####- 48 -  Contar al cantidad de personas contactada por publico objetivo.

####- 49 - Contar la cantidad de direcciones, mails y telefonos por publico objetivo.

####- 50 - Contar la cantidad de hombres y mujeres por publico objetivo.

#### - 51 - Calcular la edad promedio por publico objetivo.

# Carga de datos, generacion de archivos .parquet

####- 52 - Guardar las agregaciones generadas en un archivo parquet, particionado por mes de campaña y publico objetivo.

####- 53 - Guardar las agregaciones generadas en un archivo parquet, particionado por mes de campaña y publico objetivo.

####- 54 - Guardar las agregaciones generadas en un archivo parquet, particionado por mes de campaña y publico objetivo.

####- 55 - Guardar las agregaciones generadas en un archivo parquet, particionado por mes de campaña y publico objetivo.

#Borrador

In [94]:
#df_marital_status.show()
df_marital_ok.show(truncate=False)
#df_segment_type.show()
#df_customer_documents.show()
#df_address_type.show()
#df_gender.show()
#df_nationality.show(n=85)
#df_personal_type.show()
#df_phone_type.show()
#df_province.show()
#df_customer_segment.show()
#df_customer_info_temp.show()
#df_customer_basics.show(n=200)

+-------------------+------------------------+-------------------------+--------------------------------------------------+-------------------------+-------------------+----------------+--------------------+-----------------------+-------------------+-----------------------+-----------------+
|marital_status_type|first_record_filter_type|second_record_filter_type|martial_status_desc                               |martial_status_short_desc|registry_entry_date|last_change_date|last_change_hms_date|last_change_terminal_id|last_change_user_id|operational_load_date  |customer_end_date|
+-------------------+------------------------+-------------------------+--------------------------------------------------+-------------------------+-------------------+----------------+--------------------+-----------------------+-------------------+-----------------------+-----------------+
|0                  |                        |                         |NO INFORMA                                    